In [1]:
import os
os.chdir("../../")

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd

In [3]:
!Rscript Toy.R

Loading required package: scater
Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: methods
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.m

In [4]:
counts_df = pd.read_csv("./data/toy/toy.csv")
counts_df = counts_df.transpose()
counts_df.head()

,Gene1,Gene2,Gene3,Gene4,Gene5,Gene6,Gene7,Gene8,Gene9,Gene10,...,Gene4991,Gene4992,Gene4993,Gene4994,Gene4995,Gene4996,Gene4997,Gene4998,Gene4999,Gene5000
Cell1,2,0,0,0,0,0,0,32,3,0,...,0,0,0,119,0,0,0,0,245,0
Cell2,2,0,0,1,0,0,0,8,2,0,...,2,0,1,155,2,2,1,1,150,0
Cell3,3,0,0,0,0,1,0,26,1,0,...,1,0,0,199,8,3,8,0,104,0
Cell4,4,0,0,1,0,0,0,36,1,0,...,0,0,0,251,0,0,1,0,155,0
Cell5,2,0,0,1,0,0,0,13,1,0,...,1,0,0,157,2,2,1,0,138,0


In [5]:
batches_df = pd.read_csv("./data/toy/toy_batch.csv")
batches_df['x'].value_counts()

Batch7    4000
Batch3    4000
Batch6    4000
Batch4    4000
Batch1    4000
Batch9    4000
Batch8    4000
Batch2    4000
Batch5    4000
Name: x, dtype: int64

In [6]:
celltypes_df = pd.read_csv("./data/toy/toy_celltype.csv")
celltypes_df['x'].value_counts()

Group4    7292
Group5    7240
Group1    7203
Group2    7166
Group3    7099
Name: x, dtype: int64

In [7]:
counts_df.shape, batches_df.shape, celltypes_df.shape

((36000, 5000), (36000, 1), (36000, 1))

In [8]:
counts_adata = sc.AnnData(X=counts_df.values)
counts_adata.obs['study'] = batches_df.values
counts_adata.obs['celltype'] = celltypes_df.values
counts_adata.var_names = counts_df.columns
counts_adata

AnnData object with n_obs × n_vars = 36000 × 5000 
    obs: 'study', 'celltype'

In [9]:
counts_adata.write_h5ad("./data/toy/toy_count.h5ad")

... storing 'study' as categorical
... storing 'celltype' as categorical


In [10]:
import warnings
warnings.filterwarnings('ignore')
import surgeon as sg

Using TensorFlow backend.


In [11]:
normalized_adata = counts_adata.copy()

In [12]:
normalized_adata = sg.tl.normalize(normalized_adata, 'study', target_sum=1e4)
normalized_adata

Using 298 HVGs from full intersect set
Using 371 HVGs from n_batch-1 set
Using 369 HVGs from n_batch-2 set
Using 399 HVGs from n_batch-3 set
Using 405 HVGs from n_batch-4 set
Using 158 HVGs from n_batch-5 set
Using 2000 HVGs


AnnData object with n_obs × n_vars = 36000 × 2000 
    obs: 'study', 'celltype', 'n_counts', 'size_factors'
    var: 'n_counts'
    uns: 'log1p'

In [13]:
normalized_adata.X.min(), normalized_adata.X.max()

(0.0, 8.97027)

In [14]:
normalized_adata.write_h5ad("./data/toy/toy_normalized.h5ad")